# Unlocking Langchain Framework

In [1]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

# LLM Practice

In [9]:
from langchain.llms import OpenAI

In [11]:
llm = OpenAI(temperature=0.7)
city_name = llm.predict("I want to open a Indian restaurant in UK, give me only the best city name for that.")
print(city_name)



Birmingham, England.


In [12]:
llm("I want to open a Turkish restaurant in US, give me only the best city name for that.")

'\n\nSan Francisco, California.'

# Prompt Templates

In [16]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['culture', 'location'],
    template = "I want to open a {culture} restaurant in {location}, give me only the best city name for that."
)
prompt = prompt_template_name.format(culture="Arabic", location="UK")
print("prompt:",prompt)
print(llm.predict(prompt))

prompt: I want to open a Arabic restaurant in UK, give me only the best city name for that.


Birmingham


# Chains

In [19]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run(culture="Mexican", location="Turkey")

'\n\nIstanbul.'